In [3]:
from tqdm import tqdm
import pandas as pd

from rltensor.environments.utils import get_data


df = pd.read_json("https://poloniex.com/public?command=return24hVolume")
pairs = [pair for pair in df.columns if pair.startswith('BTC')]
print(pairs)

start = '2014-11-01 00:00:00'
# end = '2016-09-07 04:00:00'
end = '2016-10-28 08:00:00'

dfs = []
data_dir = "data"
for pair in tqdm(pairs):
    dfs.append(get_data(pair, data_dir, start, end))

  0%|          | 0/67 [00:00<?, ?it/s]

['BTC_AMP', 'BTC_ARDR', 'BTC_BCH', 'BTC_BCN', 'BTC_BCY', 'BTC_BELA', 'BTC_BLK', 'BTC_BTCD', 'BTC_BTM', 'BTC_BTS', 'BTC_BURST', 'BTC_CLAM', 'BTC_CVC', 'BTC_DASH', 'BTC_DCR', 'BTC_DGB', 'BTC_DOGE', 'BTC_EMC2', 'BTC_ETC', 'BTC_ETH', 'BTC_EXP', 'BTC_FCT', 'BTC_FLDC', 'BTC_FLO', 'BTC_GAME', 'BTC_GAS', 'BTC_GNO', 'BTC_GNT', 'BTC_GRC', 'BTC_HUC', 'BTC_LBC', 'BTC_LSK', 'BTC_LTC', 'BTC_MAID', 'BTC_NAV', 'BTC_NEOS', 'BTC_NMC', 'BTC_NXC', 'BTC_NXT', 'BTC_OMG', 'BTC_OMNI', 'BTC_PASC', 'BTC_PINK', 'BTC_POT', 'BTC_PPC', 'BTC_RADS', 'BTC_REP', 'BTC_RIC', 'BTC_SBD', 'BTC_SC', 'BTC_STEEM', 'BTC_STORJ', 'BTC_STR', 'BTC_STRAT', 'BTC_SYS', 'BTC_VIA', 'BTC_VRC', 'BTC_VTC', 'BTC_XBC', 'BTC_XCP', 'BTC_XEM', 'BTC_XMR', 'BTC_XPM', 'BTC_XRP', 'BTC_XVC', 'BTC_ZEC', 'BTC_ZRX']
Get BTC_AMP from 1414814400 to 1477656000 with period 1800
Finish.


  1%|▏         | 1/67 [00:21<23:18, 21.19s/it]

Get BTC_ARDR from 1414814400 to 1477656000 with period 1800
Finish.


  3%|▎         | 2/67 [00:41<22:29, 20.77s/it]

...

Get BTC_XVC from 1414814400 to 1477656000 with period 1800
Finish.


 99%|█████████▊| 66/67 [22:58<00:20, 20.89s/it]

Get BTC_ZEC from 1414814400 to 1477656000 with period 1800
No data.
Get BTC_ZRX from 1414814400 to 1477656000 with period 1800


100%|██████████| 67/67 [22:58<00:00, 20.58s/it]

No data.
